In [2]:
import numpy as np
import re
from gensim.utils import simple_preprocess
import pandas as pd
from nltk import flatten
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import os
from tensorflow.train import CheckpointOptions, latest_checkpoint
from tensorflow.data import Dataset
from transformers import TFAutoModel
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Dense, Dropout, concatenate
from tensorflow.keras.utils import plot_model
import numpy as np
from tensorflow.keras.preprocessing.text import tokenizer_from_json
from tensorflow.keras.models import Model
from tensorflow import keras
import json
import pickle 
from preprocess import text_preprocess

In [12]:
from transformers import TFRobertaModel, AdamWeightDecay

tf.keras.utils.get_custom_objects().update({'TFRobertaModel': TFRobertaModel})
tf.keras.utils.get_custom_objects().update({'AdamWeightDecay': AdamWeightDecay})

model = tf.keras.models.load_model('E:/KLTN/ABSA/model/bert.h5', custom_objects={"AdamWeightDecay": AdamWeightDecay})

In [15]:
def predict(model, inputs, batch_size = 1, verbose = 0):
    y_pred = model.predict(inputs, batch_size = batch_size, verbose = verbose)
    y_pred = y_pred.reshape(len(y_pred), -1, 4)
    return np.argmax(y_pred, axis=-1) # sentiment values (position that have max value)

In [16]:
def print_acsa_pred(replacements, categories, sentence_pred):
    sentiments = map(lambda x: replacements[x], sentence_pred)
    for category, sentiment in zip(categories, sentiments):
        if sentiment: print(f'=> {category},{sentiment}')

In [17]:
input_text = "quán đẹp, đồ ăn ngon nhưng nhân viên phục vụ thái độ không tốt"

In [35]:
replacements = {0: None, 1: 'positive', 2: 'negative', 3: 'neutral'}

In [36]:
categories = {'AMBIENCE#GENERAL',	'DRINKS#PRICES',	'DRINKS#QUALITY',	'DRINKS#STYLE&OPTIONS',
          'FOOD#PRICES',	'FOOD#QUALITY',	'FOOD#STYLE&OPTIONS', 'LOCATION#GENERAL',
					'RESTAURANT#GENERAL',	'RESTAURANT#MISCELLANEOUS',	'RESTAURANT#PRICES',	'SERVICE#GENERAL'}

In [47]:
PRETRAINED_MODEL = 'vinai/phobert-base'

In [48]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL)
tokenizer.max_model_input_sizes

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


{'vinai/phobert-base': 256, 'vinai/phobert-large': 256}

In [50]:
example_input = text_preprocess(input_text)
tokenized_input = tokenizer(example_input, padding='max_length', truncation=True)
features = {x: [[tokenized_input[x]]] for x in tokenizer.model_input_names}

pred = predict(model, Dataset.from_tensor_slices(features))
print_acsa_pred(replacements, categories, pred[0])

=> RESTAURANT#GENERAL,positive
=> LOCATION#GENERAL,positive


In [1]:
from vncorenlp import VnCoreNLP
rdrsegmenter = VnCoreNLP("VnCoreNLP/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m')